# MLOps stage 1 : data management

## Overview

This tutorial demonstrates how to use Vertex AI for E2E MLOps on Google Cloud in production

## Objective

In this tutorial, you learn how to use BigQuery as a dataset for training with Vertex AI.

This tutorial uses the following Google Cloud ML services:
- Vertex AI Datasets
- BigQuery Datasets

The steps performed include:
- Create a BigQuery dataset from CSV files.
- Create a Vertex AI Dataset resource from BigQuery table
- Select rows from a BigQuery dataset into a pandas dataframe -- compatible for custom training.

## Dataset

The dataset used in this example is the [Synthetic Financial Fraud dataset from Kaggle](https://www.kaggle.com/datasets/ealaxi/paysim1). PaySim simulates mobile money transactions based on a sample of real transactions extracted from one month of financial logs from a mobile money service implemented in an African country. The original logs were provided by a multinational company, who is the provider of the mobile financial service which is currently running in more than 14 countries all around the world.

### Installations

In [20]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

extra_pkgs = "google-cloud-bigquery"
! pip3 install --upgrade --quiet {USER_FLAG} google-cloud-aiplatform $extra_pkgs

In [21]:
# Restart the kernel
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Import Libraries and Define Constants

In [1]:
import google.cloud.aiplatform as aiplatform
import pandas as pd
from google.cloud import bigquery

In [2]:
REGION='us-central1'
PROJECT_ID='bq-experiments-350102'

## Initialize Vertex AI SDK for Python

In [3]:
aiplatform.init(project=PROJECT_ID, location=REGION)

## Data Dictionary

**step**: **int**: 1-743:  Maps a unit of time in the real world. In this case 1 step is 1 hour of time. <br>
**type**: **enum**: CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER <br>
**amount**: **decimal**: 0-92.4m: amount of the transaction in local currency. <br>
**nameOrig**: **string**: 6353307 unique values: customer who started the transaction. <br>
**oldBalanceOrg**: **decimal**: 0-59.6m: initial balance before the transaction. <br>
**newBalanceOrig**: **decimal**: 0-49.6m: customer's balance after the transaction. <br>
**nameDest**: **string**: 2722326 unique values: recipient ID of the transaction. <br>
**oldBalanceDest**: **decimal**: 0-356m: initial recipient balance before the transaction. <br>
**newBalanceDest**: **decimal**: 0-356m: recipient's balance after the transaction. <br>
**isFraud**: **boolean**: 0-1: identifies a fraudulent transaction (1) and non fraudulent (0)<br>

## Create BigQuery Client

In [4]:
bqclient = bigquery.Client(project=PROJECT_ID)

## Create Dataset

In [5]:
dataset_id = "bq-experiments-350102.synthetic_financial_fraud"
dataset = bigquery.Dataset(dataset_id)

dataset.location = "US"

# Send the dataset to the API for creation, with an explicit timeout.
# Raises google.api_core.exceptions.Conflict if the Dataset already
# exists within the project.
dataset = bqclient.create_dataset(dataset, timeout=30)  # Make an API request.
print("Created dataset {}.{}".format(bqclient.project, dataset.dataset_id))

Created dataset bq-experiments-350102.synthetic_financial_fraud


## Create BiqQuery Load Job

In [25]:
table_id = "bq-experiments-350102.synthetic_financial_fraud.transaction_data"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("step", "INT64", mode="REQUIRED"),
        bigquery.SchemaField("type", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("amount", "NUMERIC", mode="REQUIRED"),
        bigquery.SchemaField("nameOrig", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("oldBalanceOrg", "NUMERIC", mode="REQUIRED"),
        bigquery.SchemaField("newBalanceOrig", "NUMERIC", mode="REQUIRED"),
        bigquery.SchemaField("nameDest", "STRING", mode="REQUIRED"),
        bigquery.SchemaField("oldBalanceDest", "NUMERIC", mode="REQUIRED"),
        bigquery.SchemaField("newBalanceDest", "NUMERIC", mode="REQUIRED"),
        bigquery.SchemaField("isFraud", "BOOL", mode="REQUIRED"),
        bigquery.SchemaField("isFlaggedFraud", "BOOL", mode="REQUIRED"),
    ],
    skip_leading_rows=1,
)
uri = "gs://bq-experiments-fraud/synthetic-fraud.csv"

load_job = bqclient.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Wait for the job to complete.

table = bqclient.get_table(table_id)
print("Loaded {} rows to table {}".format(table.num_rows, table_id))

Loaded 6362620 rows to table bq-experiments-350102.synthetic_financial_fraud.transaction_data


## Select Sample into Pandas Dataframe

In [33]:
sql = """
    SELECT step,
        type, 
        amount,
        nameOrig,
        oldBalanceOrg,
        newBalanceOrig,
        nameDest,
        oldBalanceDest,
        newBalanceDest,
        isFraud,
        isFlaggedFraud
    FROM `bq-experiments-350102.synthetic_financial_fraud.transaction_data`
    LIMIT 500
"""

df = bqclient.query(sql).to_dataframe()

In [34]:
df.head()

,step,type,amount,nameOrig,oldBalanceOrg,newBalanceOrig,nameDest,oldBalanceDest,newBalanceDest,isFraud,isFlaggedFraud
0,256,DEBIT,19205.090000000,C910694199,30764.000000000,11558.910000000,C1122143037,1226937.810000000,1246142.900000000,False,False
1,256,DEBIT,42069.650000000,C970826484,3711.000000000,0E-9,C656082643,186945.200000000,229014.850000000,False,False
2,256,DEBIT,7005.400000000,C1869426134,28873.000000000,21867.600000000,C127935509,468089.060000000,475094.460000000,False,False
3,256,DEBIT,2322.550000000,C1069413307,0E-9,0E-9,C270606763,15351.280000000,17673.820000000,False,False
4,256,DEBIT,7749.440000000,C130400499,0E-9,0E-9,C679496887,731189.250000000,738938.690000000,False,False
